In [1]:
import pickle

with open('captions.p', 'rb') as f:
    captions = pickle.load(f)

with open('preds.p', 'rb') as f:
    preds = pickle.load(f)

In [2]:
import stanza
import os
HOME = os.getcwd()
print(HOME)

from stanza.pipeline.core import DownloadMethod
NLP = stanza.Pipeline('en', download_method=DownloadMethod.REUSE_RESOURCES, use_gpu=True) # to avoid downloading the models every time
# nlp is a pipeline

c:\Users\bouab\DEV\see-and-tell\cntell_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


c:\Users\bouab\DEV\see-and-tell\src


2023-07-03 14:29:23 INFO: Loading these models for language: en (English):
| Processor    | Package   |
----------------------------
| tokenize     | combined  |
| pos          | combined  |
| lemma        | combined  |
| constituency | wsj       |
| depparse     | combined  |
| sentiment    | sstplus   |
| ner          | ontonotes |

2023-07-03 14:29:25 INFO: Using device: cuda
2023-07-03 14:29:25 INFO: Loading: tokenize
2023-07-03 14:29:26 INFO: Loading: pos
2023-07-03 14:29:27 INFO: Loading: lemma
2023-07-03 14:29:27 INFO: Loading: constituency
2023-07-03 14:29:27 INFO: Loading: depparse
2023-07-03 14:29:27 INFO: Loading: sentiment
2023-07-03 14:29:28 INFO: Loading: ner
2023-07-03 14:29:28 INFO: Done loading processors!


In [3]:
import re
from _collections_abc import Sequence

def __prepare_str(string: str) -> str:

    REGEX1 = r'[^a-zA-Z\d\s-]+'
    REGEX2 = r'[^a-zA-Z\d\s]+$'
    # remove any punctuation whatsover
    string = re.sub(REGEX1, '', string)
    # add '.' to the end of the sentence
    string += '.'
    # remove '-' if it is at the end of a sentence
    res = re.sub(REGEX2, '.', string.lower().strip())
    return res

def __prepare_text(list_str: Sequence[str]) -> str:
    return " ".join([__prepare_str(s) for s in list_str]).strip()

In [4]:
text = __prepare_text(captions)
print(text)

a man in a blue shirt with the number 73 on it. a man in a blue shirt with the number 73 on it. a man in a blue shirt with the number 73 on it. a man in a blue shirt is bending over a white board that says  lets do. a man and woman stand in front of a white board that says  the word  on it. a man and a woman stand in front of a white board that says  the word  on. a man and woman are standing in front of a white board that says  c c. a man in a blue shirt with the number 13 on it. a man and a woman are standing in front of a white board that says  c  on. a man in a jacket and glasses sits in a chair in front of a bookcase that says. a man and woman are drawing on a white board in a living room. a man and woman are drawing on a white board. a woman sitting on a couch with a box of drinks on the table. a man sitting in a chair with a book shelf behind him. a man is drawing a square on a white board. a man is sitting in a chair and is wearing glasses and a jacket that says  new york. a ma

In [5]:
doc = NLP(text)
doc_sentences = [s.text.lower().strip() for s in doc.sentences]
cleaned = [__prepare_str(c) for c in captions]

In [9]:
set1, set2 = set(), set()
p1, p2 = 0, 0
while p1 < len(doc_sentences) and p2 < len(cleaned):
    if cleaned[p2] == doc_sentences[p1]:
        set1.add(p1)
        set2.add(p2)
        p1 += 1
        p2 += 1
    else:
        temp_sentence = ''
        while p2 < len(cleaned) and temp_sentence.strip() != doc_sentences[p1].strip():
            temp_sentence += f" {cleaned[p2]}"
            p2 += 1
        p1 += 1

indices_captions = sorted(list(set1))
indices_doc = sorted(list(set2))

assert len(indices_captions) == len(indices_doc)

print(len(indices_captions))

for i1, i2 in zip(indices_captions, indices_doc):
    print(cleaned[i2])
    print(doc_sentences[i1])
    print("#" * 100)


18
a man in a blue shirt with the number 73 on it.
a man in a blue shirt with the number 73 on it.
####################################################################################################
a man in a blue shirt with the number 73 on it.
a man in a blue shirt with the number 73 on it.
####################################################################################################
a man in a blue shirt with the number 73 on it.
a man in a blue shirt with the number 73 on it.
####################################################################################################
a man in a blue shirt is bending over a white board that says  lets do.
a man in a blue shirt is bending over a white board that says  lets do.
####################################################################################################
a man and woman stand in front of a white board that says  the word  on it.
a man and woman stand in front of a white board that says  the word  on it.
#########

In [8]:
from captions.captions_improved import generate_captions
caps, indices = generate_captions(captions, preds)


AssertionError: 